# 1. Loading the Data and Knocking into Shape

Import relevant modules and import initial datasets

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

Load data

In [2]:
#Load different dataseets
xls = pd.ExcelFile(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\India Premier League - Sold and Unsold -  Auction Prices Dec 2022 for 2023 season.xlsx')
SoldIPL = pd.read_excel(xls, 'Sold Dec 2022')
UnSoldIPL = pd.read_excel(xls, 'Unsold Dec 2022')


In [3]:
#Look at dataset of players sold in IPL auction in Dec 2022
SoldIPL.info()
SoldIPL.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PLAYER         204 non-null    object
 1   Country        204 non-null    object
 2   Team           204 non-null    object
 3   Type           204 non-null    object
 4   Auction_price  204 non-null    object
dtypes: object(5)
memory usage: 8.1+ KB


,PLAYER,Country,Team,Type,Auction_price
0,Deepak Chahar,India,Chennai Super Kings,Bowler,"₹14,00,00,000"
1,Ambati Rayudu,India,Chennai Super Kings,Wicket Keeper,"₹6,75,00,000"
2,Dwayne Bravo,West Indies,Chennai Super Kings,All-Rounder,"₹4,40,00,000"
3,Shivam Dube,India,Chennai Super Kings,All-Rounder,"₹4,00,00,000"
4,Chris Jordan,England,Chennai Super Kings,All-Rounder,"₹3,60,00,000"


Auction prices are in Indian rupees, and different numbering system:
One Crore = 10m Rupees = £99720 (so multiply rupees by 0.009972 to convert to pounds)
One lakh = 100,000 Rupees

In [4]:
#Remove rupee symbol
SoldIPL['Auction_price'] = SoldIPL['Auction_price'].apply(lambda x: x.replace('₹','')).apply(lambda x: x.replace(',','')).astype(int)





In [5]:
#Looks good
SoldIPL.head()



,PLAYER,Country,Team,Type,Auction_price
0,Deepak Chahar,India,Chennai Super Kings,Bowler,140000000
1,Ambati Rayudu,India,Chennai Super Kings,Wicket Keeper,67500000
2,Dwayne Bravo,West Indies,Chennai Super Kings,All-Rounder,44000000
3,Shivam Dube,India,Chennai Super Kings,All-Rounder,40000000
4,Chris Jordan,England,Chennai Super Kings,All-Rounder,36000000


In [6]:
SoldIPL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   PLAYER         204 non-null    object
 1   Country        204 non-null    object
 2   Team           204 non-null    object
 3   Type           204 non-null    object
 4   Auction_price  204 non-null    int32 
dtypes: int32(1), object(4)
memory usage: 7.3+ KB


In [7]:
#Convert auction price to pounds sterling
SoldIPL['Auc_price_pounds'] = SoldIPL['Auction_price']*0.009972
SoldIPL.rename(columns = {'Auction_price':'Auction-Base_price', 'Auc_price_pounds':'Auc-Base_price_pounds'}, inplace = True)
SoldIPL.head()

,PLAYER,Country,Team,Type,Auction-Base_price,Auc-Base_price_pounds
0,Deepak Chahar,India,Chennai Super Kings,Bowler,140000000,1396080.0
1,Ambati Rayudu,India,Chennai Super Kings,Wicket Keeper,67500000,673110.0
2,Dwayne Bravo,West Indies,Chennai Super Kings,All-Rounder,44000000,438768.0
3,Shivam Dube,India,Chennai Super Kings,All-Rounder,40000000,398880.0
4,Chris Jordan,England,Chennai Super Kings,All-Rounder,36000000,358992.0


In [8]:
#And those that weren't sold
UnSoldIPL.info()
UnSoldIPL.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   PLAYER      396 non-null    object 
 1   Country     396 non-null    object 
 2   Team        396 non-null    object 
 3   Type        396 non-null    object 
 4   Base Price  396 non-null    object 
 5   Unnamed: 5  0 non-null      float64
 6   Unnamed: 6  1 non-null      object 
dtypes: float64(1), object(6)
memory usage: 21.8+ KB


,PLAYER,Country,Team,Type,Base Price,Unnamed: 5,Unnamed: 6
0,Suresh Raina,India,Unsold,Batsman,"₹2,00,00,000",NaN,NaN
1,Steve Smith,Australia,Unsold,Batsman,"₹2,00,00,000",NaN,NaN
2,Shakib Al Hasan,Bangladesh,Unsold,All-Rounder,"₹2,00,00,000",NaN,NaN
3,Adil Rashid,England,Unsold,Bowler,"₹2,00,00,000",NaN,NaN
4,Imran Tahir,South Africa,Unsold,Bowler,"₹2,00,00,000",NaN,NaN


In [9]:
#Remove rupee symbol and convert to pounds
UnSoldIPL['Base Price'] = UnSoldIPL['Base Price'].apply(lambda x: x.replace('₹','')).apply(lambda x: x.replace(',','')).astype(int)
UnSoldIPL['Base_Price_Pounds'] = UnSoldIPL['Base Price']*0.009972
UnSoldIPL.head()

,PLAYER,Country,Team,Type,Base Price,Unnamed: 5,Unnamed: 6,Base_Price_Pounds
0,Suresh Raina,India,Unsold,Batsman,20000000,NaN,NaN,199440.0
1,Steve Smith,Australia,Unsold,Batsman,20000000,NaN,NaN,199440.0
2,Shakib Al Hasan,Bangladesh,Unsold,All-Rounder,20000000,NaN,NaN,199440.0
3,Adil Rashid,England,Unsold,Bowler,20000000,NaN,NaN,199440.0
4,Imran Tahir,South Africa,Unsold,Bowler,20000000,NaN,NaN,199440.0


In [10]:
#Rename columns and drop two unnamed ones
UnSoldIPL.rename(columns = {'Base Price':'Auction-Base_price', 'Base_Price_Pounds':'Auc-Base_price_pounds'}, inplace = True)
UnSoldIPL.drop(['Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)
UnSoldIPL.head()

,PLAYER,Country,Team,Type,Auction-Base_price,Auc-Base_price_pounds
0,Suresh Raina,India,Unsold,Batsman,20000000,199440.0
1,Steve Smith,Australia,Unsold,Batsman,20000000,199440.0
2,Shakib Al Hasan,Bangladesh,Unsold,All-Rounder,20000000,199440.0
3,Adil Rashid,England,Unsold,Bowler,20000000,199440.0
4,Imran Tahir,South Africa,Unsold,Bowler,20000000,199440.0


In [11]:
#Concatenate the sold and unsold IPL players
SoldUnSold = pd.concat([SoldIPL, UnSoldIPL], ignore_index=True)
SoldUnSold.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   PLAYER                 600 non-null    object 
 1   Country                600 non-null    object 
 2   Team                   600 non-null    object 
 3   Type                   600 non-null    object 
 4   Auction-Base_price     600 non-null    int32  
 5   Auc-Base_price_pounds  600 non-null    float64
dtypes: float64(1), int32(1), object(4)
memory usage: 25.9+ KB


In [12]:
#Find key stats and characteristics of SoldUnSold dataset
SoldUnSold.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
PLAYER,600,596,Shubham Singh,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Country,600,18,India,375,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Team,600,11,Unsold,396,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Type,600,4,All-Rounder,232,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Auction-Base_price,600.0,NaN,NaN,NaN,11779166.666667,22460593.931859,2000000.0,2000000.0,2000000.0,7500000.0,152500000.0
Auc-Base_price_pounds,600.0,NaN,NaN,NaN,117461.85,223977.042688,19944.0,19944.0,19944.0,74790.0,1520730.0


In [13]:
#Assess levels of missing data. Use pd.concat to put into tabular form
missing = pd.concat([SoldUnSold.isnull().sum(), 100 * SoldUnSold.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
PLAYER,0,0.0
Country,0,0.0
Team,0,0.0
Type,0,0.0
Auction-Base_price,0,0.0
Auc-Base_price_pounds,0,0.0


In [14]:
#Bring in the other datasets
IPLBatStat = pd.read_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\IPL - BATTING ALL TIME .csv')
IPLBowlStat = pd.read_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\ipl BOWLING ALL TIME.xlsx')
IPLMVPStat = pd.read_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\ipl - most valuable player - 2022 season.xlsx')
IntCrickSals = pd.read_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\International Mens Cricket Central Contract Salaries - 2021 22.xlsx')


When look through different IPL datasets below, looks like it could be worth renaming POS columns to indicate whether batting, bowling, MVP, as is a ranking system, and is another measure of worth or value.

In [15]:
#All time batting stats for IPL - data types look good, but lots of blank rows
IPLBatStat.info()
IPLBatStat.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POS     126 non-null    float64
 1   Player  126 non-null    object 
 2   Mat     126 non-null    float64
 3   Inns    126 non-null    float64
 4   NO      126 non-null    float64
 5   Runs    126 non-null    float64
 6   HS      126 non-null    object 
 7   Avg     126 non-null    float64
 8   BF      126 non-null    float64
 9   SR      126 non-null    float64
 10  100     126 non-null    float64
 11  50      126 non-null    float64
 12  4s      126 non-null    float64
 13  6s      126 non-null    float64
dtypes: float64(12), object(2)
memory usage: 55.4+ KB


,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1.0,Virat Kohli,223.0,215.0,32.0,6624.0,113,36.20,5129.0,129.15,5.0,44.0,578.0,218.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,Shikhar Dhawan,206.0,205.0,27.0,6244.0,106*,35.08,4942.0,126.35,2.0,47.0,701.0,136.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.0,David Warner,162.0,162.0,22.0,5881.0,126,42.01,4180.0,140.69,4.0,54.0,561.0,211.0


In [16]:
#And thus lot of missing data
missing = pd.concat([IPLBatStat.isnull().sum(), 100 * IPLBatStat.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
POS,379,75.049505
Player,379,75.049505
Mat,379,75.049505
Inns,379,75.049505
NO,379,75.049505
Runs,379,75.049505
HS,379,75.049505
Avg,379,75.049505
BF,379,75.049505
SR,379,75.049505


In [17]:
#Drop rows with missing values
IPLBatStat.dropna(inplace=True)

In [18]:
#Looks better
IPLBatStat.head(10)


,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1.0,Virat Kohli,223.0,215.0,32.0,6624.0,113,36.20,5129.0,129.15,5.0,44.0,578.0,218.0
5,2.0,Shikhar Dhawan,206.0,205.0,27.0,6244.0,106*,35.08,4942.0,126.35,2.0,47.0,701.0,136.0
9,3.0,David Warner,162.0,162.0,22.0,5881.0,126,42.01,4180.0,140.69,4.0,54.0,561.0,211.0
13,4.0,Rohit Sharma,227.0,222.0,28.0,5879.0,109*,30.30,4526.0,129.89,1.0,40.0,519.0,240.0
17,5.0,Suresh Raina,205.0,200.0,30.0,5528.0,100*,32.52,4042.0,136.76,1.0,39.0,506.0,203.0
21,6.0,AB de Villiers,184.0,170.0,40.0,5162.0,133*,39.70,3403.0,151.68,3.0,40.0,413.0,251.0
25,7.0,MS Dhoni,234.0,206.0,79.0,4978.0,84*,39.20,3682.0,135.20,0.0,24.0,346.0,229.0
29,8.0,Chris Gayle,142.0,141.0,16.0,4965.0,175*,39.72,3333.0,148.96,6.0,31.0,405.0,357.0
33,9.0,Robin Uthappa,205.0,197.0,17.0,4952.0,88,27.51,3799.0,130.35,0.0,27.0,481.0,182.0
37,10.0,Dinesh Karthik,229.0,208.0,45.0,4376.0,97*,26.85,3299.0,132.65,0.0,19.0,406.0,125.0


In [19]:
#And right number of players, etc
IPLBatStat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126 entries, 0 to 501
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POS     126 non-null    float64
 1   Player  126 non-null    object 
 2   Mat     126 non-null    float64
 3   Inns    126 non-null    float64
 4   NO      126 non-null    float64
 5   Runs    126 non-null    float64
 6   HS      126 non-null    object 
 7   Avg     126 non-null    float64
 8   BF      126 non-null    float64
 9   SR      126 non-null    float64
 10  100     126 non-null    float64
 11  50      126 non-null    float64
 12  4s      126 non-null    float64
 13  6s      126 non-null    float64
dtypes: float64(12), object(2)
memory usage: 14.8+ KB


In [20]:
#Rest index
IPLBatStat = IPLBatStat.reset_index(drop = True)
IPLBatStat.head(10)

,POS,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1.0,Virat Kohli,223.0,215.0,32.0,6624.0,113,36.20,5129.0,129.15,5.0,44.0,578.0,218.0
1,2.0,Shikhar Dhawan,206.0,205.0,27.0,6244.0,106*,35.08,4942.0,126.35,2.0,47.0,701.0,136.0
2,3.0,David Warner,162.0,162.0,22.0,5881.0,126,42.01,4180.0,140.69,4.0,54.0,561.0,211.0
3,4.0,Rohit Sharma,227.0,222.0,28.0,5879.0,109*,30.30,4526.0,129.89,1.0,40.0,519.0,240.0
4,5.0,Suresh Raina,205.0,200.0,30.0,5528.0,100*,32.52,4042.0,136.76,1.0,39.0,506.0,203.0
5,6.0,AB de Villiers,184.0,170.0,40.0,5162.0,133*,39.70,3403.0,151.68,3.0,40.0,413.0,251.0
6,7.0,MS Dhoni,234.0,206.0,79.0,4978.0,84*,39.20,3682.0,135.20,0.0,24.0,346.0,229.0
7,8.0,Chris Gayle,142.0,141.0,16.0,4965.0,175*,39.72,3333.0,148.96,6.0,31.0,405.0,357.0
8,9.0,Robin Uthappa,205.0,197.0,17.0,4952.0,88,27.51,3799.0,130.35,0.0,27.0,481.0,182.0
9,10.0,Dinesh Karthik,229.0,208.0,45.0,4376.0,97*,26.85,3299.0,132.65,0.0,19.0,406.0,125.0


In [21]:
#But no missing data now
missing = pd.concat([IPLBatStat.isnull().sum(), 100 * IPLBatStat.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count')

,count,%
POS,0,0.0
Player,0,0.0
Mat,0,0.0
Inns,0,0.0
NO,0,0.0
Runs,0,0.0
HS,0,0.0
Avg,0,0.0
BF,0,0.0
SR,0,0.0


In [22]:
IPLBatStat.rename(columns = {'POS':'IPL_Bat_Rank'}, inplace = True)


In [23]:
IPLBatStat.head()

,IPL_Bat_Rank,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s
0,1.0,Virat Kohli,223.0,215.0,32.0,6624.0,113,36.20,5129.0,129.15,5.0,44.0,578.0,218.0
1,2.0,Shikhar Dhawan,206.0,205.0,27.0,6244.0,106*,35.08,4942.0,126.35,2.0,47.0,701.0,136.0
2,3.0,David Warner,162.0,162.0,22.0,5881.0,126,42.01,4180.0,140.69,4.0,54.0,561.0,211.0
3,4.0,Rohit Sharma,227.0,222.0,28.0,5879.0,109*,30.30,4526.0,129.89,1.0,40.0,519.0,240.0
4,5.0,Suresh Raina,205.0,200.0,30.0,5528.0,100*,32.52,4042.0,136.76,1.0,39.0,506.0,203.0


It's also worth noting that HS (highest score) in this batting table is an object type, presumably because some numbers have an asterisk next to them, indicating that the highest score was "not out", so usually seen as having a bit more worth/value, due to the batsman not having lost his wicket. Therefore, will need to create another column to indicate whether or not the high score was not out. Will then have to apply this to other batting tables.

In [24]:
HS_was_NO = []
for score in IPLBatStat['HS']:
    if '*' in score:
        HS_was_NO.append(True)
    else:
        HS_was_NO.append(False)
IPLBatStat['HS_was_NO'] = HS_was_NO
IPLBatStat.head()

,IPL_Bat_Rank,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s,HS_was_NO
0,1.0,Virat Kohli,223.0,215.0,32.0,6624.0,113,36.20,5129.0,129.15,5.0,44.0,578.0,218.0,False
1,2.0,Shikhar Dhawan,206.0,205.0,27.0,6244.0,106*,35.08,4942.0,126.35,2.0,47.0,701.0,136.0,True
2,3.0,David Warner,162.0,162.0,22.0,5881.0,126,42.01,4180.0,140.69,4.0,54.0,561.0,211.0,False
3,4.0,Rohit Sharma,227.0,222.0,28.0,5879.0,109*,30.30,4526.0,129.89,1.0,40.0,519.0,240.0,True
4,5.0,Suresh Raina,205.0,200.0,30.0,5528.0,100*,32.52,4042.0,136.76,1.0,39.0,506.0,203.0,True


In [25]:
#Remove asterisk from High Score and convert type to float64
IPLBatStat['HS'] = IPLBatStat['HS'].apply(lambda x: x.replace('*','')).astype(float)
IPLBatStat.info()
IPLBatStat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   IPL_Bat_Rank  126 non-null    float64
 1   Player        126 non-null    object 
 2   Mat           126 non-null    float64
 3   Inns          126 non-null    float64
 4   NO            126 non-null    float64
 5   Runs          126 non-null    float64
 6   HS            126 non-null    float64
 7   Avg           126 non-null    float64
 8   BF            126 non-null    float64
 9   SR            126 non-null    float64
 10  100           126 non-null    float64
 11  50            126 non-null    float64
 12  4s            126 non-null    float64
 13  6s            126 non-null    float64
 14  HS_was_NO     126 non-null    bool   
dtypes: bool(1), float64(13), object(1)
memory usage: 14.0+ KB


,IPL_Bat_Rank,Player,Mat,Inns,NO,Runs,HS,Avg,BF,SR,100,50,4s,6s,HS_was_NO
0,1.0,Virat Kohli,223.0,215.0,32.0,6624.0,113.0,36.20,5129.0,129.15,5.0,44.0,578.0,218.0,False
1,2.0,Shikhar Dhawan,206.0,205.0,27.0,6244.0,106.0,35.08,4942.0,126.35,2.0,47.0,701.0,136.0,True
2,3.0,David Warner,162.0,162.0,22.0,5881.0,126.0,42.01,4180.0,140.69,4.0,54.0,561.0,211.0,False
3,4.0,Rohit Sharma,227.0,222.0,28.0,5879.0,109.0,30.30,4526.0,129.89,1.0,40.0,519.0,240.0,True
4,5.0,Suresh Raina,205.0,200.0,30.0,5528.0,100.0,32.52,4042.0,136.76,1.0,39.0,506.0,203.0,True


In [26]:
#Make HS-NO changes into a function
def HS_NO_col(df, colname):
    '''Function to check which high scores were not-out scores
    and then create column with boolean True/False to indicate, 
    so that originaL HS col can have asterisks removed 
    and data type changed to float. Takes two arguments: dataframe 
    working on, and column name'''
    HS_was_NO = []
    for score in colname:
        if '*' in score:
            HS_was_NO.append(True)
        else:
            HS_was_NO.append(False)
    df['HS_was_NO'] = HS_was_NO
    df['colname'] = df['colname'].apply(lambda x: x.replace('*','')).astype(float)

In [27]:
#All time bowling data from IPL - same issues as with batting data
IPLBowlStat.info()
IPLBowlStat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 668 entries, 0 to 667
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   POS     168 non-null    float64
 1   Player  168 non-null    object 
 2   Mat     168 non-null    float64
 3   Inns    168 non-null    float64
 4   Ov      168 non-null    float64
 5   Runs    168 non-null    float64
 6   Wkts    168 non-null    float64
 7   BBI     168 non-null    object 
 8   Avg     168 non-null    float64
 9   Econ    168 non-null    float64
 10  SR      168 non-null    float64
dtypes: float64(9), object(2)
memory usage: 57.5+ KB


,POS,Player,Mat,Inns,Ov,Runs,Wkts,BBI,Avg,Econ,SR
0,1.0,Dwayne Bravo,161.0,158.0,516.0,4359.0,183.0,2022-04-01 00:00:00,23.82,8.38,17.05
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
IPLBowlStat.dropna(inplace=True)
IPLBowlStat = IPLBowlStat.reset_index(drop = True)
IPLBowlStat.head(10)

,POS,Player,Mat,Inns,Ov,Runs,Wkts,BBI,Avg,Econ,SR
0,1.0,Dwayne Bravo,161.0,158.0,516.0,4359.0,183.0,2022-04-01 00:00:00,23.82,8.38,17.05
1,2.0,Lasith Malinga,122.0,122.0,471.0,3366.0,170.0,2013-05-01 00:00:00,19.80,7.14,16.63
2,3.0,Amit Mishra,154.0,154.0,540.0,3980.0,166.0,2017-05-01 00:00:00,23.98,7.36,19.55
3,4.0,Yuzvendra Chahal,131.0,130.0,475.0,3624.0,166.0,1940-05-01 00:00:00,21.83,7.61,17.20
4,5.0,Piyush Chawla,165.0,164.0,545.0,4301.0,157.0,2017-04-01 00:00:00,27.39,7.88,20.85
5,6.0,Ravichandran Ashwin,184.0,181.0,649.0,4535.0,157.0,1934-04-01 00:00:00,28.89,6.98,24.84
6,7.0,Bhuvneshwar Kumar,146.0,146.0,542.0,3971.0,154.0,2019-05-01 00:00:00,25.79,7.30,21.18
7,8.0,Sunil Narine,148.0,147.0,576.0,3820.0,152.0,2019-05-01 00:00:00,25.13,6.63,22.75
8,9.0,Harbhajan Singh,163.0,160.0,569.0,4030.0,150.0,2018-05-01 00:00:00,26.87,7.08,22.77
9,10.0,Jasprit Bumrah,120.0,120.0,455.0,3378.0,145.0,2023-10-05 00:00:00,23.30,7.39,18.91


In [29]:
IPLBowlStat.rename(columns = {'POS':'IPL_Bowl_Rank'}, inplace = True)
IPLBowlStat.head()

,IPL_Bowl_Rank,Player,Mat,Inns,Ov,Runs,Wkts,BBI,Avg,Econ,SR
0,1.0,Dwayne Bravo,161.0,158.0,516.0,4359.0,183.0,2022-04-01 00:00:00,23.82,8.38,17.05
1,2.0,Lasith Malinga,122.0,122.0,471.0,3366.0,170.0,2013-05-01 00:00:00,19.80,7.14,16.63
2,3.0,Amit Mishra,154.0,154.0,540.0,3980.0,166.0,2017-05-01 00:00:00,23.98,7.36,19.55
3,4.0,Yuzvendra Chahal,131.0,130.0,475.0,3624.0,166.0,1940-05-01 00:00:00,21.83,7.61,17.20
4,5.0,Piyush Chawla,165.0,164.0,545.0,4301.0,157.0,2017-04-01 00:00:00,27.39,7.88,20.85


In [30]:
#Most valued player ratings, with same issues
IPLMVPStat.info()
IPLMVPStat.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   POS        172 non-null    float64
 1   Player     172 non-null    object 
 2   Pts        172 non-null    float64
 3   Mat        172 non-null    float64
 4   Wkts       172 non-null    float64
 5   Dots       172 non-null    float64
 6   4s         172 non-null    float64
 7   6s         172 non-null    float64
 8   Catches    172 non-null    float64
 9   Run outs   172 non-null    float64
 10  Stumpings  172 non-null    float64
dtypes: float64(10), object(1)
memory usage: 59.1+ KB


,POS,Player,Pts,Mat,Wkts,Dots,4s,6s,Catches,Run outs,Stumpings
0,1.0,Jos Buttler,387.5,17.0,0.0,0.0,83.0,45.0,9.0,0.0,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
IPLMVPStat.dropna(inplace=True)
IPLMVPStat = IPLMVPStat.reset_index(drop = True)
IPLMVPStat.head(10)

,POS,Player,Pts,Mat,Wkts,Dots,4s,6s,Catches,Run outs,Stumpings
0,1.0,Jos Buttler,387.5,17.0,0.0,0.0,83.0,45.0,9.0,0.0,0.0
1,2.0,Hardik Pandya,284.5,15.0,8.0,76.0,49.0,12.0,4.0,6.0,0.0
2,3.0,Andre Russell,281.0,14.0,17.0,58.0,18.0,32.0,2.0,1.5,0.0
3,4.0,Prasidh Krishna,273.0,17.0,19.0,200.0,0.0,0.0,2.0,1.5,0.0
4,5.0,Liam Livingstone,265.5,14.0,6.0,38.0,29.0,34.0,6.0,0.0,0.0
5,6.0,Wanindu Hasaranga,258.0,16.0,26.0,143.0,4.0,1.0,3.0,3.0,0.0
6,7.0,Mohammed Shami,252.5,16.0,20.0,172.0,0.0,0.0,3.0,3.0,0.0
7,8.0,Rashid Khan,250.0,16.0,19.0,127.0,3.0,9.0,7.0,0.0,0.0
8,9.0,Trent Boult,245.5,16.0,16.0,176.0,2.0,1.0,2.0,0.0,0.0
9,10.0,Sanju Samson,244.5,17.0,0.0,0.0,43.0,26.0,14.0,6.0,2.0


In [32]:
IPLMVPStat.rename(columns = {'POS':'IPL_MVP_Rank'}, inplace = True)
IPLMVPStat.head()

,IPL_MVP_Rank,Player,Pts,Mat,Wkts,Dots,4s,6s,Catches,Run outs,Stumpings
0,1.0,Jos Buttler,387.5,17.0,0.0,0.0,83.0,45.0,9.0,0.0,0.0
1,2.0,Hardik Pandya,284.5,15.0,8.0,76.0,49.0,12.0,4.0,6.0,0.0
2,3.0,Andre Russell,281.0,14.0,17.0,58.0,18.0,32.0,2.0,1.5,0.0
3,4.0,Prasidh Krishna,273.0,17.0,19.0,200.0,0.0,0.0,2.0,1.5,0.0
4,5.0,Liam Livingstone,265.5,14.0,6.0,38.0,29.0,34.0,6.0,0.0,0.0


In [33]:
#Need to do same with Pts col
IPLMVPStat.rename(columns = {'Pts':'IPL_MVP_Pts'}, inplace = True)
IPLMVPStat.head()

,IPL_MVP_Rank,Player,IPL_MVP_Pts,Mat,Wkts,Dots,4s,6s,Catches,Run outs,Stumpings
0,1.0,Jos Buttler,387.5,17.0,0.0,0.0,83.0,45.0,9.0,0.0,0.0
1,2.0,Hardik Pandya,284.5,15.0,8.0,76.0,49.0,12.0,4.0,6.0,0.0
2,3.0,Andre Russell,281.0,14.0,17.0,58.0,18.0,32.0,2.0,1.5,0.0
3,4.0,Prasidh Krishna,273.0,17.0,19.0,200.0,0.0,0.0,2.0,1.5,0.0
4,5.0,Liam Livingstone,265.5,14.0,6.0,38.0,29.0,34.0,6.0,0.0,0.0


In [34]:
IntCrickSals.info()
IntCrickSals.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 15 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Players                                176 non-null    object 
 1   Country                                176 non-null    object 
 2   Retainer Fee / contract / base salary  165 non-null    object 
 3   Test Fee                               171 non-null    object 
 4   ODI Fee                                167 non-null    object 
 5   T20 Fee                                164 non-null    object 
 6   Unnamed: 6                             0 non-null      float64
 7   Unnamed: 7                             0 non-null      float64
 8   Unnamed: 8                             0 non-null      float64
 9   Unnamed: 9                             0 non-null      float64
 10  Unnamed: 10                            0 non-null      float64
 11  Unname

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,Babar Azam,Pakistan,1.25 million PKR (+ undisclosed Captain allowa...,"PKR 50,000","PKR 25,000","PKR 15,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,One Crore = 10m Rupees
1,Mohammad Rizwan,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"One lakh = 100,000 Rupees"
2,Hasan Ali,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Shaheen Afridi,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
#Drop unnamed columns
IntCrickSals.drop(['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14'], axis=1, inplace=True)
IntCrickSals.head()


,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
0,Babar Azam,Pakistan,1.25 million PKR (+ undisclosed Captain allowa...,"PKR 50,000","PKR 25,000","PKR 15,000"
1,Mohammad Rizwan,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
2,Hasan Ali,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
3,Shaheen Afridi,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",NaN,NaN


In [36]:
IntCrickSals.head(25)

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
0,Babar Azam,Pakistan,1.25 million PKR (+ undisclosed Captain allowa...,"PKR 50,000","PKR 25,000","PKR 15,000"
1,Mohammad Rizwan,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
2,Hasan Ali,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
3,Shaheen Afridi,Pakistan,1.25 million PKR per month,"PKR 50,000","PKR 25,000","PKR 15,000"
4,Azhar Ali,Pakistan,"937,500 PKRper month","PKR 40,000",NaN,NaN
5,Faheem Ashraf,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500"
6,Fakhar Zaman,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500"
7,Fawad Alam,Pakistan,"937,500 PKR per month","PKR 40,000",NaN,NaN
8,Shadab Khan,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000","PKR 12,500"
9,Yasir Shah,Pakistan,"937,500 PKR per month","PKR 40,000","PKR 20,000",NaN


In [37]:
IntCrickSals.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Players                                176 non-null    object
 1   Country                                176 non-null    object
 2   Retainer Fee / contract / base salary  165 non-null    object
 3   Test Fee                               171 non-null    object
 4   ODI Fee                                167 non-null    object
 5   T20 Fee                                164 non-null    object
dtypes: object(6)
memory usage: 8.4+ KB


In [38]:
#How many countries are represented in dataset? Turns out to be 9
IntCrickSals.Country.unique()

array(['Pakistan', 'India', 'England', 'Australia', 'South Africa',
       'New Zealand', 'West Indies', 'Sri Lanka', 'Bangladesh'],
      dtype=object)

In [39]:
#View India players only - double-checked values and Jasprit Bumrah in partic - 
#the India base salary figures are annual, and JB's is a mistake to say monthly
India_players = IntCrickSals[IntCrickSals["Country"] == "India"]
print(India_players)

                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                             7 Crores    
21     Rohit Sharma (C)   India                             7 Crores    
22       Jasprit Bumrah   India                    7 Crores per month   
23  Ravichandran Ashwin   India                             5 Crores    
24      Ravindra Jadeja   India                             5 Crores    
25         Rishabh Pant   India                              5 Crores   
26        KL Rahul (VC)   India                              5 Crores   
27       Mohammad Shami   India                              5 Crores   
28    Cheteshwar Pujara   India                              3 Crores   
29       Ajinkya Rahane   India                              3 Crores   
30           Axer Patel   India                              3 Crores   
31       Shradul Thakur   India                              3 Crores   
32         Sheryas Iyer   India                    

In [40]:
England_players = IntCrickSals[IntCrickSals["Country"] == "England"]
print(England_players)

              Players  Country          Retainer Fee / contract / base salary  \
47        Eoin Morgan  England  £925,000 (plus undisclosed captain allowance)   
48     James Anderson  England                                         925000   
49       Stuart Broad  England                                         925000   
50           Joe Root  England  £925,000 (Undisclosed test captain allowance)   
51         Ben Stokes  England                                         925000   
52  Jonathan Bairstow  England                                         925000   
53        Jos Buttler  England                                         925000   
54       Chris Woakes  England                                         925000   
55          Moeen Ali  England                                         925000   
56         Sam Curran  England                                         925000   
57       Jofra Archer  England                                         925000   
58          Jason Roy  Engla

In [41]:
Bangla_players = IntCrickSals[IntCrickSals["Country"] == "Bangladesh"]
Bangla_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
155,Tamim Iqbal,Bangladesh,"$57,000","$7,100","$3,600","$2,400"
156,Soumya Sarkar,Bangladesh,NaN,"$7,100","$3,600","$2,400"
157,Shakib Al Hasan,Bangladesh,"$57,000","$7,100","$3,600","$2,400"
158,Shadman Islam,Bangladesh,NaN,"$7,100","$3,600","$2,400"
159,Sabbir Rahman,Bangladesh,NaN,"$7,100","$3,600","$2,400"


In [42]:
Aust_players = IntCrickSals[IntCrickSals["Country"] == "Australia"]
Aust_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
67,Aaron Finch,Australia,$1.5 million,"$15,000","$18,000","$20,000"
68,Tim Paine,Australia,$1 million,"$15,000","$18,000","$20,000"
69,Steve Smith,Australia,$1 million,"$15,000","$18,000","$20,000"
70,Pat Cummins,Australia,"$850,000","$15,000","$18,000","$20,000"
71,Glenn Maxwell,Australia,"$850,000","$15,000","$18,000","$20,000"


In [43]:
WI_players = IntCrickSals[IntCrickSals["Country"] == "West Indies"]
WI_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
119,Jason Holder,West Indies,$80000,"$6,500","$4,500","$3,000"
120,Kraigg Brathwaite,West Indies,$60000,"$6,500","$4,500","$3,000"
121,Jermaine Blackwood,West Indies,$60000,"$6,500","$4,500","$3,000"
122,Nkrumah Bonner,West Indies,$60000,"$6,500","$4,500","$3,000"
123,Rahkeem Cornwall,West Indies,$60000,"$6,500","$4,500","$3,000"


In [44]:
SouthA_players = IntCrickSals[IntCrickSals["Country"] == "South Africa"]
SouthA_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
84,Temba Bavuma,South Africa,"$100,000","$4,500","$1,200",$800
85,Quinton de Kock,South Africa,"$350,000","$4,500","$1,200",$800
86,Dean Elgar,South Africa,"$100,000","$4,500","$1,200",$800
87,Beuran Hendricks,South Africa,"$100,000","$4,500","$1,200",$800
88,Reeza Hendricks,South Africa,"$100,000","$4,500","$1,200",$800


In [45]:
NZ_players = IntCrickSals[IntCrickSals["Country"] == "New Zealand"]
NZ_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
100,Kane Williamson (C),New Zealand,"$440,000","$8,495","$3,682","$2,407"
101,Martin Guptill,New Zealand,"$350,000",NaN,"$3,682","$2,407"
102,Colin Munro,New Zealand,"$180,000",NaN,NaN,"$2,407"
103,Henry Nicholls,New Zealand,"$85,585","$8,495","$3,682",NaN
104,Ross Taylor,New Zealand,"$400,000","$8,495",NaN,"$2,407"


In [46]:
SriL_players = IntCrickSals[IntCrickSals["Country"] == "Sri Lanka"]
SriL_players.head()

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
137,Dimuth Karunaratne,Sri Lanka,"$100,000","$7,500","$5,500","$3,500"
138,Dinesh Chandimal,Sri Lanka,"$80,000","$7,500","$5,500","$3,500"
139,Dasun Shanaka,Sri Lanka,"$65,000","$7,500","$5,500","$3,500"
140,Suranga Lakmal,Sri Lanka,"$60,000","$7,500","$5,500","$3,500"
141,Lasith Embuldeniya,Sri Lanka,"$60,000","$7,500","$5,500","$3,500"


Looked at different salary levels - default currency is US dollars, but pounds for England, In Rupees for India, Pakistan rupees for Pakistan (and Pakistan also monthly, compared to annual for all others) 1.25milPKR = 4932
1 In Crore = £99590, 1 PKR = 0.0035

In [47]:
IntCrickSals = IntCrickSals.fillna(0)
IntCrickSals[IntCrickSals["Country"] == "Bangladesh"]




,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
155,Tamim Iqbal,Bangladesh,"$57,000","$7,100","$3,600","$2,400"
156,Soumya Sarkar,Bangladesh,0,"$7,100","$3,600","$2,400"
157,Shakib Al Hasan,Bangladesh,"$57,000","$7,100","$3,600","$2,400"
158,Shadman Islam,Bangladesh,0,"$7,100","$3,600","$2,400"
159,Sabbir Rahman,Bangladesh,0,"$7,100","$3,600","$2,400"
160,Rubel Hossain,Bangladesh,"$43,000","$7,100","$3,600","$2,400"
161,Nazmul Hossain Shanto,Bangladesh,0,"$7,100","$3,600","$2,400"
162,Mushfiqur Rahim,Bangladesh,"$57,000","$7,100","$3,600","$2,400"
163,Mosaddek Hossain,Bangladesh,"$14,000","$7,100","$3,600","$2,400"
164,Mominul Haque,Bangladesh,"$30,000","$7,100","$3,600","$2,400"


In [48]:
IntCrickSals[IntCrickSals["Country"] == "New Zealand"]

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
100,Kane Williamson (C),New Zealand,"$440,000","$8,495","$3,682","$2,407"
101,Martin Guptill,New Zealand,"$350,000",0,"$3,682","$2,407"
102,Colin Munro,New Zealand,"$180,000",0,0,"$2,407"
103,Henry Nicholls,New Zealand,"$85,585","$8,495","$3,682",0
104,Ross Taylor,New Zealand,"$400,000","$8,495",0,"$2,407"
105,Daryl Mitchell,New Zealand,"$85,585",0,"$3,682","$2,407"
106,Tom Latham,New Zealand,"$205,266","$8,495","$3,682",0
107,Tom Blundell,New Zealand,"$85,585","$8,495","$3,682","$2,407"
108,Tim Seifert,New Zealand,"$50,000",0,0,"$2,407"
109,Bradley John Watling,New Zealand,"$50,000","$8,495",0,0


In [49]:
#IntCrickSals['Retainer Fee / contract / base salary'] = IntCrickSals['Retainer Fee / contract / base salary'].astype(str)
#IntCrickSals['Retainer Fee / contract / base salary'] = IntCrickSals['Retainer Fee / contract / base salary'].apply(lambda x: x.replace('Crores','0000000'))
IntCrickSals['Retainer Fee / contract / base salary'] = IntCrickSals['Retainer Fee / contract / base salary'].str.replace('Crores','0000000')
print(IntCrickSals[IntCrickSals["Country"] == "India"])

                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                            7 0000000    
21     Rohit Sharma (C)   India                            7 0000000    
22       Jasprit Bumrah   India                   7 0000000 per month   
23  Ravichandran Ashwin   India                            5 0000000    
24      Ravindra Jadeja   India                            5 0000000    
25         Rishabh Pant   India                             5 0000000   
26        KL Rahul (VC)   India                             5 0000000   
27       Mohammad Shami   India                             5 0000000   
28    Cheteshwar Pujara   India                             3 0000000   
29       Ajinkya Rahane   India                             3 0000000   
30           Axer Patel   India                             3 0000000   
31       Shradul Thakur   India                             3 0000000   
32         Sheryas Iyer   India                    

In [50]:
IntCrickSals['Test Fee'] = IntCrickSals['Test Fee'].str.replace('lakh','00000')
print(IntCrickSals[IntCrickSals["Country"] == "India"])

                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                            7 0000000    
21     Rohit Sharma (C)   India                            7 0000000    
22       Jasprit Bumrah   India                   7 0000000 per month   
23  Ravichandran Ashwin   India                            5 0000000    
24      Ravindra Jadeja   India                            5 0000000    
25         Rishabh Pant   India                             5 0000000   
26        KL Rahul (VC)   India                             5 0000000   
27       Mohammad Shami   India                             5 0000000   
28    Cheteshwar Pujara   India                             3 0000000   
29       Ajinkya Rahane   India                             3 0000000   
30           Axer Patel   India                             3 0000000   
31       Shradul Thakur   India                             3 0000000   
32         Sheryas Iyer   India                    

In [51]:
IntCrickSals['ODI Fee'] = IntCrickSals['ODI Fee'].str.replace('lakh','00000')
IntCrickSals['T20 Fee'] = IntCrickSals['T20 Fee'].str.replace('Lakh', '00000')
print(IntCrickSals[IntCrickSals["Country"] == "India"])

                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                            7 0000000    
21     Rohit Sharma (C)   India                            7 0000000    
22       Jasprit Bumrah   India                   7 0000000 per month   
23  Ravichandran Ashwin   India                            5 0000000    
24      Ravindra Jadeja   India                            5 0000000    
25         Rishabh Pant   India                             5 0000000   
26        KL Rahul (VC)   India                             5 0000000   
27       Mohammad Shami   India                             5 0000000   
28    Cheteshwar Pujara   India                             3 0000000   
29       Ajinkya Rahane   India                             3 0000000   
30           Axer Patel   India                             3 0000000   
31       Shradul Thakur   India                             3 0000000   
32         Sheryas Iyer   India                    

Looked up Deepak Chahar and Mayank Agarwal - mistake in their data - both on 15 lakh per test - https://www.timesofsports.com/cricket/indian-players-salary/

In [52]:
IntCrickSals.loc[45, 'Test Fee'] = 1500000
IntCrickSals.loc[46, 'Test Fee'] = 1500000
print(IntCrickSals[IntCrickSals["Country"] == "India"])

                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                            7 0000000    
21     Rohit Sharma (C)   India                            7 0000000    
22       Jasprit Bumrah   India                   7 0000000 per month   
23  Ravichandran Ashwin   India                            5 0000000    
24      Ravindra Jadeja   India                            5 0000000    
25         Rishabh Pant   India                             5 0000000   
26        KL Rahul (VC)   India                             5 0000000   
27       Mohammad Shami   India                             5 0000000   
28    Cheteshwar Pujara   India                             3 0000000   
29       Ajinkya Rahane   India                             3 0000000   
30           Axer Patel   India                             3 0000000   
31       Shradul Thakur   India                             3 0000000   
32         Sheryas Iyer   India                    

In [53]:
IntCrickSals.loc[21, 'ODI Fee'] = 720000
IntCrickSals.loc[21, 'T20 Fee'] = 360000
print(IntCrickSals[IntCrickSals["Country"] == "India"])

                Players Country Retainer Fee / contract / base salary  \
20          Virat Kohli   India                            7 0000000    
21     Rohit Sharma (C)   India                            7 0000000    
22       Jasprit Bumrah   India                   7 0000000 per month   
23  Ravichandran Ashwin   India                            5 0000000    
24      Ravindra Jadeja   India                            5 0000000    
25         Rishabh Pant   India                             5 0000000   
26        KL Rahul (VC)   India                             5 0000000   
27       Mohammad Shami   India                             5 0000000   
28    Cheteshwar Pujara   India                             3 0000000   
29       Ajinkya Rahane   India                             3 0000000   
30           Axer Patel   India                             3 0000000   
31       Shradul Thakur   India                             3 0000000   
32         Sheryas Iyer   India                    

In [54]:
IntCrickSals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 6 columns):
 #   Column                                 Non-Null Count  Dtype 
---  ------                                 --------------  ----- 
 0   Players                                176 non-null    object
 1   Country                                176 non-null    object
 2   Retainer Fee / contract / base salary  147 non-null    object
 3   Test Fee                               151 non-null    object
 4   ODI Fee                                147 non-null    object
 5   T20 Fee                                144 non-null    object
dtypes: object(6)
memory usage: 8.4+ KB


In [57]:
null_series = pd.isnull(IntCrickSals['Retainer Fee / contract / base salary'])
IntCrickSals[null_series]

,Players,Country,Retainer Fee / contract / base salary,Test Fee,ODI Fee,T20 Fee
48,James Anderson,England,NaN,NaN,NaN,NaN
49,Stuart Broad,England,NaN,NaN,NaN,NaN
51,Ben Stokes,England,NaN,NaN,NaN,NaN
52,Jonathan Bairstow,England,NaN,NaN,NaN,NaN
53,Jos Buttler,England,NaN,NaN,NaN,NaN
54,Chris Woakes,England,NaN,NaN,NaN,NaN
55,Moeen Ali,England,NaN,NaN,NaN,NaN
56,Sam Curran,England,NaN,NaN,NaN,NaN
57,Jofra Archer,England,NaN,NaN,NaN,NaN
58,Jason Roy,England,NaN,NaN,NaN,NaN


In [ ]:
IntCrickSals['Base_sal_pounds'] = IntCrickSals['Retainer Fee / contract / base salary']
IntCrickSals.head()

In [ ]:
IntCrickSals = IntCrickSals.fillna(0)
IntCrickSals.head()

In [ ]:
IntCrickSals['Base_sal_pounds'] = IntCrickSals['Base_sal_pounds'].str.replace('(\D+)', '')
#(' ', '').str.replace('$', '').str.replace(',' '')
IntCrickSals.head()

In [ ]:
IntCrickSals.info()

In [ ]:
#As seems to be indicating null values in Base_sal_pounds, and so can't change data type to integer, need to investigate further
null_series = pd.isnull(IntCrickSals['Base_sal_pounds'])

IntCrickSals[null_series]
#IntCrickSals.Base_sal_pounds = IntCrickSals.Base_sal_pounds.astype(int)
#print(IntCrickSals.dtypes)

In [ ]:



#NOTE - got stuck here trying to remove text from some of numbers cols in order to 
#convert everything into pounds - will leave for now to research and concentrate on joining 
#some of previous tables

#So strip out extraneous data, convert to integers and convert to pounds (to match earlier data)


#IntCrickSals[['Base_sal_pounds', 'Test_Fee_pounds', 'ODI_Fee_pounds', 'T20_Fee_pounds']] = 
#IntCrickSals[['Retainer Fee / contract / base salary', 'Test Fee', 'ODI Fee', 'T20 Fee']].replace(r'\D+', '').astype(int)
#IntCrickSals.head()
#for fee in IntCrickSals['Base_sal_pounds']:
 #   if IntCrickSals['Country'] == "Pakistan":
  #      if IntCrickSals['Players'] == 'Babar Azam' or IntCrickSals['Players'] == 'Mohammad Rizwan' or IntCrickSals['Players'] == 'Hasan Ali' or IntCrickSals['Players'] == 'Shaheen Afridi':
   #         IntCrickSals['Base_sal_pounds'] = 59184
    #    else:
     #       IntCrickSals['Base_sal_pounds'] = IntCrickSals['Base_sal_pounds']*12*0.0035
    #elif IntCrickSals['Country'] == "India":
     #   IntCrickSals['Base_sal_pounds'] = IntCrickSals['Base_sal_pounds']*99590
    #elif IntCrickSals['Country'] == 'England':
     #   IntCrickSals['Base_sal_pounds'] = IntCrickSals['Base_sal_pounds']
    #elif IntCrickSals['Country'] == 'Australia':
     #   if IntCrickSals['Players'] == 'Steve Smith' or IntCrickSals['Players'] == 'Tim Paine':
      #      IntCrickSals['Base_sal_pounds'] = 807025
       # elif IntCrickSals['Players'] == 'Aaron Paine':
        #    IntCrickSals['Base_sal_pounds'] = 1210537
  #  else:
   #     IntCrickSals['Base_sal_pounds'] = IntCrickSals['Base_sal_pounds']*0.81
            
        

In [ ]:
SoldUnSold.rename(columns = {'PLAYER':'Player'}, inplace = True)
SoldUnSold.head()

In [ ]:
IPLPriceNStat1 = SoldUnSold.merge(IPLBatStat, on='Player', how='left')
IPLPriceNStat1.info()
IPLPriceNStat1.head()


In [ ]:
IPLPriceNStat2 = IPLPriceNStat1.merge(IPLBowlStat, on='Player', how='left')
IPLPriceNStat2.info()
IPLPriceNStat2.head()

Could consider adding MPV ratings here, but had trouble joining them. Need to do further exploration
Joining has only brought across a fraction of those in all time stats, presumably because many of those not now playing
Need to consider way to search and include players who may not be playing this year, but could be in future years. 
Perhaps this could be supplied by the full international T20 data.
Let's bring that in now, so we have it to hand


In [ ]:
IntT20Bat = pd.read_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\International Mens Cricket Stats\Batting\t20.csv')
IntT20Bowl = pd.read_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\International Mens Cricket Stats\Bowling\Bowling_t20.csv')
IntT20Field = pd.read_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\raw_data\International Mens Cricket Stats\Fielding\Fielding_t20.csv')


In [ ]:
IntT20Bat.info()
IntT20Bat.head()

In [ ]:
IntT20Bowl.info()
IntT20Bowl.head()

In [ ]:
IntT20Field.info()
IntT20Field.head()

If need to use this data, will need to work on naming of players col as this won't automatically match with IPL listings

In [ ]:
IPLPriceNStat2.to_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\IPLPriceNStatcleaneddata.xlsx')
IPLBatStat.to_csv(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\IPLBatStatscleaneddata.csv')
IPLBowlStat.to_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\IPLBowlStatscleandata.xlsx')
IPLMVPStat.to_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\IPLMVPStatscleaneddata.xlsx')
IntCrickSals.to_excel(r'C:\Users\Pearc\OneDrive\Documents\Data Science\Springboard\SpringboardCapstone2\data\interim_data\InternatCrickSalspartcleaneddata.xlsx')
